# e-footprint quickstart

This notebook provides an example scenario that you can use to get familiar with the Python API of efootprint: the daily video consumption of all French households on a big streaming platform.

You will get to describe:

- the infrastructure involved (servers with auto-scaling settings, storage and network)
- the user journey involving 2 steps (Streaming, Upload)
- the usage pattern and the device population that executes it (the laptops of all French households)

## Import the packages

⚠ If this steps fails, remember to run *ipython kernel install --user --name=efootprint-kernel* _inside_ your python virtual environement (initializable with `poetry install`) to be able to select efootprint-kernel as the jupyter kernel.

In [1]:
# If this hasn’t been done in virtualenv (useful for Google colab notebook)
!pip install efootprint


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

from efootprint.abstract_modeling_classes.source_objects import SourceValue, Sources, SourceObject
from efootprint.abstract_modeling_classes.explainable_objects import EmptyExplainableObject
from efootprint.core.usage.user_journey import UserJourney
from efootprint.core.usage.user_journey_step import UserJourneyStep
from efootprint.core.usage.job import Job
from efootprint.core.hardware.server import Server, ServerTypes
from efootprint.core.hardware.storage import Storage
from efootprint.core.usage.usage_pattern import UsagePattern
from efootprint.core.hardware.network import Network
from efootprint.core.system import System
from efootprint.constants.countries import Countries
from efootprint.constants.units import u
from efootprint.utils.object_relationships_graphs import USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE

## Define the infrastructure

### Creating objects manually
An e-footprint object has a name and attributes describing its technical and environmental characteristics:

In [3]:
storage = Storage(
    "SSD storage",
    carbon_footprint_fabrication_per_storage_capacity=SourceValue(160 * u.kg / u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    power_per_storage_capacity=SourceValue(1.3 * u.W / u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    lifespan=SourceValue(6 * u.years, Sources.HYPOTHESIS),
    idle_power=SourceValue(0 * u.W, Sources.HYPOTHESIS),
    storage_capacity=SourceValue(1 * u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    data_replication_factor=SourceValue(3 * u.dimensionless, Sources.HYPOTHESIS),
    data_storage_duration = SourceValue(2 * u.year, Sources.HYPOTHESIS),
    base_storage_need = SourceValue(100 * u.TB, Sources.HYPOTHESIS),
    fixed_nb_of_instances = EmptyExplainableObject()
    )

### Creating objects from default values

All e-footprint classes also implement default values and a from_defaults method that allows for using a set a pre-defined default attributes and specifying the ones we want to specify through keyword arguments.

In [4]:
Storage.default_values()

{'carbon_footprint_fabrication_per_storage_capacity': 160 kilogram,
 'power_per_storage_capacity': 1.3 watt,
 'lifespan': 6 year,
 'idle_power': 0 watt,
 'storage_capacity': 1 terabyte,
 'data_replication_factor': 3 dimensionless,
 'base_storage_need': 0 terabyte,
 'data_storage_duration': 5 year}

In [5]:
# Creating a storage object from defaults while specifying storage capacity using keyword arguments
print(Storage.from_defaults("2 TB SSD storage", storage_capacity=SourceValue(2 * u.TB)))

Storage id-450dd0-2-TB-SSD-storage
 
name: 2 TB SSD storage
lifespan: 6 year
fraction_of_usage_time: 1 dimensionless
carbon_footprint_fabrication_per_storage_capacity: 160 kilogram
power_per_storage_capacity: 1.3 watt
idle_power: 0 watt
storage_capacity: 2 terabyte
data_replication_factor: 3 dimensionless
data_storage_duration: 5 year
base_storage_need: 0 terabyte
fixed_nb_of_instances: no value
 
calculated_attributes:
  carbon_footprint_fabrication: 0 kilogram
  power: 0 watt
  storage_delta: no value
  full_cumulative_storage_need: no value
  raw_nb_of_instances: no value
  nb_of_instances: no value
  nb_of_active_instances: no value
  instances_fabrication_footprint: no value
  instances_energy: no value
  energy_footprint: no value



We can see from the above print that e-footprint objects have calculated attributes that are setup as None and then computed by e-footprint when the modeling is over. More information on e-footprint objects’ calculated_attributes can be found in the [e-footprint documentation](https://boavizta.github.io/e-footprint/).

### Creating objects from archetypes

Some e-footprint objects (Storage, Network and Hardware) also have archetypes that have their own set of default values:

In [6]:
Storage.archetypes()

[<bound method Storage.ssd of <class 'efootprint.core.hardware.storage.Storage'>>,
 <bound method Storage.hdd of <class 'efootprint.core.hardware.storage.Storage'>>]

In [7]:
print(Storage.hdd())

Storage id-cd41c2-Default-HDD-storage
 
name: Default HDD storage
lifespan: 4 year
fraction_of_usage_time: 1 dimensionless
carbon_footprint_fabrication_per_storage_capacity: 20.0 kilogram / gigabyte
power_per_storage_capacity: 4.2 watt / gigabyte
idle_power: 0 watt
storage_capacity: 1 terabyte
data_replication_factor: 3 dimensionless
data_storage_duration: 5 year
base_storage_need: 0 terabyte
fixed_nb_of_instances: no value
 
calculated_attributes:
  carbon_footprint_fabrication: 0 kilogram
  power: 0 watt
  storage_delta: no value
  full_cumulative_storage_need: no value
  raw_nb_of_instances: no value
  nb_of_instances: no value
  nb_of_active_instances: no value
  instances_fabrication_footprint: no value
  instances_energy: no value
  energy_footprint: no value



Apart from environmental and technical attributes, e-footprint objects can link to other e-footprint objects. For example, server objects have a storage attribute:

In [8]:
server = Server.from_defaults(
    "server",
    server_type=ServerTypes.autoscaling(),
    power_usage_effectiveness=SourceValue(1.2 * u.dimensionless, Sources.HYPOTHESIS),
    average_carbon_intensity=SourceValue(100 * u.g / u.kWh, Sources.HYPOTHESIS),
    server_utilization_rate=SourceValue(0.9 * u.dimensionless, Sources.HYPOTHESIS),
    base_ram_consumption=SourceValue(300 * u.MB, Sources.HYPOTHESIS),
    base_compute_consumption=SourceValue(2 * u.cpu_core, Sources.HYPOTHESIS),
    storage=storage
)

print(server)

Server id-642867-server
 
name: server
carbon_footprint_fabrication: 600 kilogram
power: 300 watt
lifespan: 6 year
fraction_of_usage_time: 1 dimensionless
server_type: autoscaling
idle_power: 50 watt
ram: 128 gigabyte
compute: 24 cpu_core
power_usage_effectiveness: 1.2 dimensionless
average_carbon_intensity: 100.0 gram / kilowatt_hour
server_utilization_rate: 0.9 dimensionless
base_ram_consumption: 300 megabyte
base_compute_consumption: 2 cpu_core
fixed_nb_of_instances: no value
storage: id-58ba69-SSD-storage
 
calculated_attributes:
  hour_by_hour_ram_need: no value
  hour_by_hour_compute_need: no value
  occupied_ram_per_instance: no value
  occupied_compute_per_instance: no value
  available_ram_per_instance: no value
  available_compute_per_instance: no value
  raw_nb_of_instances: no value
  nb_of_instances: no value
  instances_fabrication_footprint: no value
  instances_energy: no value
  energy_footprint: no value



### Creating objects from builders connected to external data sources

Of course only relying on a single set of default values for creating our servers won’t get us far. That’s why e-footprint provides a builder class that connects to [Boavizta’s API](https://github.com/Boavizta/boaviztapi) to allow for the creation of servers from a cloud provider and an instance type.

In [9]:
from efootprint.builders.hardware.boavizta_cloud_server import BoaviztaCloudServer

# Some attributes can only take specific values
for attribute, attribute_list_value in BoaviztaCloudServer.list_values().items():
    print(f"Possible values for {attribute}: {attribute_list_value}")

Possible values for server_type: [autoscaling, on-premise, serverless]
Possible values for provider: [aws, azure, scaleway]


In [10]:
# Moreover, some attributes depend on another attribute for their values
for attribute, attribute_conditional_dict in BoaviztaCloudServer.conditional_list_values().items():
    condition_attribute = attribute_conditional_dict['depends_on']
    print(f"Possible values for {attribute} depend on {condition_attribute}:\n")
    for condition_value, possible_values in attribute_conditional_dict["conditional_list_values"].items():
        if len(possible_values) > 10:
            values_to_print = possible_values[:5] + ["etc."]
        else:
            values_to_print = possible_values
        print(f"    Possible values when {condition_attribute} is {condition_value}: {values_to_print}")
    print("\n")

Possible values for fixed_nb_of_instances depend on server_type:

    Possible values when server_type is autoscaling: [no value]
    Possible values when server_type is serverless: [no value]


Possible values for instance_type depend on provider:

    Possible values when provider is aws: [a1.medium, a1.large, a1.xlarge, a1.2xlarge, a1.4xlarge, 'etc.']
    Possible values when provider is azure: [d2ads_v5, d4ads_v5, d8ads_v5, d16ads_v5, d32ads_v5, 'etc.']
    Possible values when provider is scaleway: [coparm1-16c-64g, coparm1-2c-8g, coparm1-32c-128g, coparm1-4c-16g, coparm1-8c-32g, 'etc.']




In [11]:
# BoaviztaCloudServer still has quite a lot of default values but ones that are much easier to make hypothesis on, 
# like lifespan, server utilisation rate or power usage effectiveness
default_values = BoaviztaCloudServer.default_values()
default_values.pop("storage")
default_values

{'provider': scaleway,
 'instance_type': dev1-s,
 'server_type': autoscaling,
 'average_carbon_intensity': 0.23 kilogram / kilowatt_hour,
 'lifespan': 6 year,
 'idle_power': 0 watt,
 'power_usage_effectiveness': 1.2 dimensionless,
 'server_utilization_rate': 0.9 dimensionless,
 'base_ram_consumption': 0 gigabyte,
 'base_compute_consumption': 0 cpu_core,
 'fixed_nb_of_instances': no value}

In [12]:
# The most difficult environmental and technical attributes are retrieved from a call to BoaviztAPI:
print(BoaviztaCloudServer.from_defaults("Default Boavizta cloud server"))

2025-01-28 12:19:04,552 - INFO - Computing calculated attributes for BoaviztaCloudServer Default Boavizta cloud server
2025-01-28 12:19:04,552 - INFO - Calling Boavizta API with url https://api.boavizta.org/v1/cloud/instance, method GET and params {'params': {'provider': 'scaleway', 'instance_type': 'dev1-s'}}


BoaviztaCloudServer id-70b46a-Default-Boavizta-cloud-server
 
name: Default Boavizta cloud server
lifespan: 6 year
fraction_of_usage_time: 1 dimensionless
server_type: autoscaling
idle_power: 0 watt
power_usage_effectiveness: 1.2 dimensionless
average_carbon_intensity: 0.23 kilogram / kilowatt_hour
server_utilization_rate: 0.9 dimensionless
base_ram_consumption: 0 gigabyte
base_compute_consumption: 0 cpu_core
fixed_nb_of_instances: no value
storage: id-33bbb7-Default-SSD-storage
provider: scaleway
instance_type: dev1-s
 
calculated_attributes:
  api_call_response: {'impacts': {'gwp': {'unit': 'kgCO2eq', 'description': 'Tota...
  carbon_footprint_fabrication: 20.0 kilogram
  power: 11.33 watt
  ram: 256.0 gigabyte
  compute: 16.0 cpu_core
  hour_by_hour_ram_need: no value
  hour_by_hour_compute_need: no value
  occupied_ram_per_instance: 0 gigabyte
  occupied_compute_per_instance: 0 cpu_core
  available_ram_per_instance: 230.4 gigabyte
  available_compute_per_instance: 14.4 cpu_core
  r

## [Optional] Install services on your server

Manually creating job objects can get tricky because you have to specify how much RAM and compute the job uses on the server it runs on during its duration. That’s why e-footprint allows for the installation of services on servers, that will give access to higher-level job classes that compute these very technical attributes from simpler ones. For example, let’s install a video streaming service on our server:

### Video streaming service

In [13]:
from efootprint.builders.services.video_streaming import VideoStreaming

VideoStreaming.default_values()

{'base_ram_consumption': 2 gigabyte,
 'bits_per_pixel': 0.1 dimensionless,
 'static_delivery_cpu_cost': 4.0 cpu_core * second / gigabyte,
 'ram_buffer_per_user': 50 megabyte}

In [14]:
video_streaming_service = VideoStreaming.from_defaults("Video streaming service", server=server)

2025-01-28 12:19:04,790 - INFO - Computing calculated attributes for VideoStreaming Video streaming service


In [15]:
# All services have a list of compatible job types, let’s check out the ones for video streaming:
VideoStreaming.compatible_jobs

<bound method Service.compatible_jobs of <class 'efootprint.builders.services.video_streaming.VideoStreaming'>>

In [16]:
# There’s only one so let’s use it !
from efootprint.builders.services.video_streaming import VideoStreamingJob

VideoStreamingJob.default_values()

{'resolution': 1080p (1920 x 1080),
 'video_duration': 1 hour,
 'refresh_rate': 30.0 / second}

In [17]:
print(VideoStreamingJob.list_values())

{'resolution': [480p (640 x 480), 720p (1280 x 720), 1080p (1920 x 1080), 1440p (2560 x 1440), 2K (2048 x 1080), 4K (3840 x 2160), 8K (7680 x 4320)]}


In [18]:
# Now it’s easy to add a 1 hour 1080p streaming job to our streaming service
streaming_job = VideoStreamingJob(
    "streaming job", service=video_streaming_service, resolution=SourceObject("1080p (1920 x 1080)"), 
    video_duration=SourceValue(1 * u.hour), refresh_rate=SourceValue(30 / u.s))

# For optimization purposes calculations are only made when usage data has been entered but we can force
# some of them to see what the VideoStreamingJob does.
streaming_job.update_dynamic_bitrate()
streaming_job.update_data_download()
streaming_job.update_compute_needed()
streaming_job.update_ram_needed()

print(streaming_job)

VideoStreamingJob id-5b5d41-streaming-job
 
name: streaming job
data_transferred: 0 kilobyte
data_stored: 0 kilobyte
request_duration: 1 hour
service: id-07d853-Video-streaming-service
resolution: 1080p (1920 x 1080)
refresh_rate: 30 / second
 
calculated_attributes:
  dynamic_bitrate: 0.78 megabyte / second
  data_download: 2.8 gigabyte
  compute_needed: 0.0 cpu_core
  ram_needed: 50 megabyte
  hourly_occurrences_per_usage_pattern: {}
  hourly_avg_occurrences_per_usage_pattern: {}
  hourly_data_transferred_per_usage_pattern: {}
  hourly_data_download_per_usage_pattern: {}
  hourly_data_stored_per_usage_pattern: {}
  hourly_occurrences_across_usage_patterns: no value
  hourly_avg_occurrences_across_usage_patterns: no value
  hourly_data_transferred_across_usage_patterns: no value
  hourly_data_stored_across_usage_patterns: no value



### Web application service

In the same vein, we can install a web application service on our server. e-footprint’s WebApplication service relies on the analysis of [Boavizta’s ecobenchmark project](https://github.com/Boavizta/ecobenchmark-applicationweb-backend).

In [25]:
from efootprint.builders.services.web_application import WebApplication, WebApplicationJob

web_app_service = WebApplication("Web app", server=server, technology=SourceObject("php-symfony"))
web_app_job = WebApplicationJob.from_defaults("fetching web view", service=web_app_service)
web_app_job.update_compute_needed()
web_app_job.update_ram_needed()

print(web_app_service)
print(web_app_job)

2025-01-28 12:27:56,213 - INFO - Computing calculated attributes for WebApplication Web app


WebApplication id-9f4f99-Web-app
 
name: Web app
server: id-642867-server
base_ram_consumption: no value
base_compute_consumption: no value
technology: php-symfony

WebApplicationJob id-c46ac5-fetching-web-view
 
name: fetching web view
data_transferred: 200 kilobyte
data_download: 2 megabyte
data_stored: 100 kilobyte
request_duration: 1 second
service: id-9f4f99-Web-app
implementation_details: default
 
calculated_attributes:
  compute_needed: 0.08 cpu_core
  ram_needed: 6.15 megabyte
  hourly_occurrences_per_usage_pattern: {}
  hourly_avg_occurrences_per_usage_pattern: {}
  hourly_data_transferred_per_usage_pattern: {}
  hourly_data_download_per_usage_pattern: {}
  hourly_data_stored_per_usage_pattern: {}
  hourly_occurrences_across_usage_patterns: no value
  hourly_avg_occurrences_across_usage_patterns: no value
  hourly_data_transferred_across_usage_patterns: no value
  hourly_data_stored_across_usage_patterns: no value



## Define the user journey

This is the modeling of the average daily usage of the streaming platform in France:

In [19]:
streaming_step = UserJourneyStep(
    "20 min streaming",
    user_time_spent=SourceValue(20 * u.min, Sources.USER_DATA),
    jobs=[
        Job(
            "streaming",
            server=server,
            data_transferred=SourceValue(0.05 * u.MB, Sources.USER_DATA),
            data_download=SourceValue(800 * u.MB, Sources.USER_DATA),
            data_stored=SourceValue(0.05 * u.MB, Sources.USER_DATA),
            request_duration=SourceValue(4 * u.min, Sources.HYPOTHESIS),
            cpu_needed=SourceValue(1 * u.core, Sources.HYPOTHESIS),
            ram_needed=SourceValue(50 * u.MB, Sources.HYPOTHESIS)
            )
        ]
    )
upload_step = UserJourneyStep(
    "1 min video capture then upload",
    user_time_spent=SourceValue(70 * u.s, Sources.USER_DATA),
    jobs=[
        Job(
            "video upload",
            server=server,
            data_transferred=SourceValue(20 * u.MB, Sources.USER_DATA),
            data_download=SourceValue(0 * u.GB, Sources.USER_DATA),
            data_stored=SourceValue(20 * u.MB, Sources.USER_DATA),
            request_duration=SourceValue(2 * u.s, Sources.HYPOTHESIS),
            cpu_needed=SourceValue(1 * u.core, Sources.HYPOTHESIS),
            ram_needed=SourceValue(50 * u.MB, Sources.HYPOTHESIS)
        )
    ]
)

UndefinedUnitError: 'core' is not defined in the unit registry

The user journey is then simply a list of user journey steps:

In [ ]:
user_journey = UserJourney("Mean video consumption user journey", uj_steps=[streaming_step, upload_step])

## Describe usage

An e-footprint usage pattern links a user journey to devices that run it, a network, a country, and the number of times the user journey gets executed hour by hour. 

In [ ]:
# Let’s build synthetic usage data by summing a linear growth with a sinusoidal fluctuation components, then adding daily variation
from datetime import datetime, timedelta

from efootprint.builders.time_builders import linear_growth_hourly_values

start_date = datetime.strptime("2025-01-01", "%Y-%m-%d")
timespan = 3 * u.year

linear_growth = linear_growth_hourly_values(timespan, start_value=5000, end_value=100000, start_date=start_date)
linear_growth.set_label("Hourly user journeys linear growth component")

linear_growth.plot()

In [ ]:
from efootprint.builders.time_builders import sinusoidal_fluct_hourly_values

sinusoidal_fluct = sinusoidal_fluct_hourly_values(
    timespan, sin_fluct_amplitude=3000, sin_fluct_period_in_hours=3 * 30 * 24, start_date=start_date)

lin_growth_plus_sin_fluct = (linear_growth + sinusoidal_fluct).set_label("Hourly user journeys linear growth with sinusoidal fluctuations")

lin_growth_plus_sin_fluct.plot()

In [ ]:
# Let’s add daily variations because people use the system less at night
from efootprint.builders.time_builders import daily_fluct_hourly_values

daily_fluct = daily_fluct_hourly_values(timespan, fluct_scale=0.8, hour_of_day_for_min_value=4, start_date=start_date)
daily_fluct.set_label("Daily volume fluctuation")

daily_fluct.plot(xlims=[start_date, start_date+timedelta(days=1)])

In [ ]:
hourly_user_journey_starts = lin_growth_plus_sin_fluct * daily_fluct
hourly_user_journey_starts.set_label("Hourly number of user journey started")

hourly_user_journey_starts.plot(xlims=[start_date, start_date + timedelta(days=7)])

In [ ]:
# Over 3 years the daily fluctuations color the area between daily min and max number of hourly user journeys
hourly_user_journey_starts.plot()

In [ ]:
network = Network(
        "WIFI network",
        bandwidth_energy_intensity=SourceValue(0.05 * u("kWh/GB"), Sources.TRAFICOM_STUDY))

usage_pattern = UsagePattern(
    "Daily video streaming consumption",
    user_journey=user_journey,
    devices=[default_laptop()],
    network=network,
    country=Countries.FRANCE(),
    hourly_user_journey_starts=hourly_user_journey_starts
)

system = System("System", usage_patterns=[usage_pattern])

## Results

### Computed attributes

Now all calculated_attributes have been computed:

In [ ]:
print(server)

### System footprint overview

In [ ]:
system.plot_footprints_by_category_and_object("System footprints.html")

### Object relationships graph

Hover over a node to get the numerical values of its environmental and technical attributes. For simplifying the graph the Network and Hardware nodes are not shown.

In [ ]:
usage_pattern.object_relationship_graph_to_file("object_relationships_graph.html", width="800px", height="500px",
    classes_to_ignore=USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE, notebook=True)

### Calculus graph

Any e-footprint calculation can generate its calculation graph for full auditability. Hover on a calculus node to display its formula and numeric value.

In [ ]:
usage_pattern.devices_fabrication_footprint.calculus_graph_to_file(
    "device_population_fab_footprint_calculus_graph.html", width="800px", height="500px", notebook=True)

### Plotting an object’s hourly and cumulated CO2 emissions

In [ ]:
server.energy_footprint.plot()

In [ ]:
server.energy_footprint.plot(cumsum=True)

In [ ]:
system.total_footprint.plot(cumsum=True)

## Analysing the impact of a change
### Numeric input change
Any input change automatically triggers the computation of calculations that depend on the input. For example, let’s say that the average data download consumption of the streaming step decreased because of a change in default video quality:

In [ ]:
streaming_step.jobs[0].data_download = SourceValue(500 * u.MB, Sources.USER_DATA)

In [ ]:
system.plot_emission_diffs("bandwith reduction.png")

### System structure change
Now let’s make a more complex change, like adding a conversation with a generative AI chatbot before streaming the video.
Numerical values don’t matter so much for the sake of this tutorial, please check out <a href="https://github.com/publicissapient-france/e-footprint-modelings" target="_blank">the e-footprint-modelings github repository</a> for a more detailed modeling of the impact of LLM training and inference.

In [ ]:
llm_server = Autoscaling(
    "Inference GPU server",
    carbon_footprint_fabrication=SourceValue(4900 * u.kg, Sources.HYPOTHESIS),
    power=SourceValue(6400 * u.W, Sources.HYPOTHESIS),
    lifespan=SourceValue(5 * u.year, Sources.HYPOTHESIS),
    idle_power=SourceValue(500 * u.W, Sources.HYPOTHESIS),
    ram=SourceValue(128 * u.GB, Sources.HYPOTHESIS),
    cpu_cores=SourceValue(16 * u.core, Sources.HYPOTHESIS), # Used to represent GPUs because e-footprint doesn’t natively model GPU resources yet.
    power_usage_effectiveness=SourceValue(1.2 * u.dimensionless, Sources.HYPOTHESIS),
    average_carbon_intensity=SourceValue(300 * u.g / u.kWh, Sources.HYPOTHESIS),
    server_utilization_rate=SourceValue(1 * u.dimensionless, Sources.HYPOTHESIS),
    base_ram_consumption=SourceValue(0 * u.MB, Sources.HYPOTHESIS),
    base_cpu_consumption=SourceValue(0 * u.core, Sources.HYPOTHESIS),
    storage=default_ssd()
)

In [ ]:
llm_chat_step = UserJourneyStep(
    "Chat with LLM to select video", user_time_spent=SourceValue(1 * u.min, Sources.HYPOTHESIS),
    jobs=[Job("LLM API call", llm_server, SourceValue(300 * u.kB, Sources.USER_DATA),
              SourceValue(300 * u.kB, Sources.USER_DATA), SourceValue(300 * u.kB, Sources.USER_DATA),
              request_duration=SourceValue(5 * u.s, Sources.HYPOTHESIS),
              cpu_needed=SourceValue(16 * u.core, Sources.HYPOTHESIS),
              ram_needed=SourceValue(128 * u.GB, Sources.HYPOTHESIS))])

In [ ]:
# Adding the new step is simply an attribute update.
user_journey.uj_steps.append(llm_chat_step)

In [ ]:
system.plot_emission_diffs("LLM chat addition.png")

We can see that server energy footprint has been multiplied by more than 10 and the rest of the impact is quite negligible. Good to know to make informed decisions ! Of course the impact is very much dependent on assumptions. If the LLM server ran on low-carbon electricity for example:

In [ ]:
llm_server.average_carbon_intensity = SourceValue(50 * u.g / u.kWh, Sources.HYPOTHESIS)
system.plot_emission_diffs("lower LLM inference carbon intensity.png")

## Recap of all System changes

In [ ]:
system.plot_emission_diffs("All system diffs.png", from_start=True)

## Making simulations of changes in the future

We’ve seen that you can make changes in your modeling and analyse the differences, but most likely the changes you’re contemplating will happen at some point in the future. Let’s model a change in the future thanks to e-footprint’s ModelingUpdate object !

In [ ]:
# Let’s first revert the system to its state before changes
# We can make optimized batch changes by using the ModelingUpdate object, that is also used to make simulations of changes in the future
from efootprint.abstract_modeling_classes.modeling_update import ModelingUpdate

ModelingUpdate([
    [user_journey.uj_steps, [streaming_step, upload_step]],
    [llm_server.average_carbon_intensity, SourceValue(300 * u.g / u.kWh, Sources.HYPOTHESIS)],
    [streaming_step.jobs[0].data_download, SourceValue(800 * u.MB, Sources.USER_DATA)]
])

system.plot_footprints_by_category_and_object("System footprints after reset.html")

In [ ]:
# To create a simulation, which is a change in the future, simply set ModelingUpdate’s simulation_date parameter
simulation = ModelingUpdate([[user_journey.uj_steps, [streaming_step, upload_step, llm_chat_step]]],
                           simulation_date=start_date + timedelta(days=365))

In [ ]:
system.total_footprint.plot(cumsum=True)

In [ ]:
llm_server.energy_footprint.plot(cumsum=True)

In [ ]:
# The system state is reset to baseline after the simulation.
# For example, our LLM server has no energy footprint since it is not used in the baseline modeling
llm_server.energy_footprint

In [ ]:
# To set simulation values, use ModelingUpdate’s set_updated_values method
simulation.set_updated_values()
llm_server.energy_footprint

In [ ]:
# Conversely, pre-update values are reset using ModelingUpdate’s reset_values method
simulation.reset_values()
llm_server.energy_footprint